In [0]:
import numpy as np 
import pandas as pd 
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.tokenize import word_tokenize
import seaborn as sns
import matplotlib.pyplot as plt

In [31]:
import requests
from pprint import pprint

locationUrlFromLatLong = "https://developers.zomato.com/api/v2.1/geocode?lat=42.8781&lon=-87.6333"
header = {"User-agent": "curl/7.43.0", "Accept": "application/json", "user_key": "API_Key"}

response = requests.get(locationUrlFromLatLong, headers=header)

#pprint(response.json())
var1 = response.json()
data1 = (var1['nearby_restaurants'])
print (data1)

[{'restaurant': {'R': {'has_menu_status': {'delivery': -1, 'takeaway': -1}, 'res_id': 17176104}, 'apikey': 'a6e08b68522003610f9d16344ae7c0e3', 'id': '17176104', 'name': "Infusino's Pizzeria", 'url': 'https://www.zomato.com/milwaukee/infusinos-pizzeria-racine?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1', 'location': {'address': '3301 Washington Avenue, Racine 53405', 'locality': 'Racine', 'city': 'Milwaukee', 'city_id': 1267, 'latitude': '42.7185770000', 'longitude': '-87.8186800000', 'zipcode': '53405', 'country_id': 216, 'locality_verbose': 'Racine, Milwaukee'}, 'switch_to_order_menu': 0, 'cuisines': 'Pizza', 'average_cost_for_two': 25, 'price_range': 2, 'currency': '$', 'offers': [], 'opentable_support': 0, 'is_zomato_book_res': 0, 'mezzo_provider': 'OTHER', 'is_book_form_web_view': 0, 'book_form_web_view_url': '', 'book_again_url': '', 'thumb': '', 'user_rating': {'aggregate_rating': '3.9', 'rating_text': 'Good', 'rating_color': '9ACD32', 'rating_obj': {'title': {'tex

In [0]:
df = pd.DataFrame(columns=['id',"name", "address","locality","city","url",'cuisines','cost_2','photo','rating_no','rating_text'])


In [0]:
df.drop(df.index, inplace=True)

for res in data1:

  df = df.append({
      'id': res['restaurant']['id'],
     "name": res['restaurant']['name'],
     "address":res['restaurant']['location']['address'],
     "locality":res['restaurant']['location']['locality'],
     "city":res['restaurant']['location']['city'],
     "url":res['restaurant']['url'],
     'cuisines':res['restaurant']['cuisines'],
     'cost_2':res['restaurant']['average_cost_for_two'],
     'photo':res['restaurant']['photos_url'],
     'rating_no':res['restaurant']['user_rating']['aggregate_rating'],
     'rating_text': res['restaurant']['user_rating']['rating_text']
      }, ignore_index=True)

In [54]:
print(df.locality)

0    Racine
1    Racine
2    Racine
3    Racine
4    Racine
5    Racine
6    Racine
7    Racine
8    Racine
Name: locality, dtype: object


In [5]:
data = pd.read_csv('zomato.csv', encoding='latin-1')


FileNotFoundError: ignored

In [0]:
#data['City'].value_counts(dropna = False)
#data_city =data.loc[data['City'] == 'New Delhi']

In [0]:
#data_new_delphi=data_city[['Restaurant Name','Cuisines','Locality','Aggregate rating']]
#data_new_delphi['Locality'].value_counts(dropna = False).head(5)

In [0]:
#data_new_delphi.loc[data['Locality'] == 'Connaught Place']

In [0]:
df1 = df[['name', 'cuisines', 'locality','rating_no']].copy()

In [61]:
print(df1)

                          name        cuisines locality rating_no
0          Infusino's Pizzeria           Pizza   Racine       3.9
1            Bernie's Pizzeria           Pizza   Racine       3.9
2                   Lee's Deli        Sandwich   Racine       3.8
3        Wilson's Coffee & Tea  Coffee and Tea   Racine       3.7
4     DeRango's The Pizza King           Pizza   Racine       3.7
5            Bendtsen's Bakery                   Racine       3.7
6                Larsen Bakery                   Racine       3.4
7                   McDonald's       Fast Food   Racine       3.1
8  River Run Family Restaurant        American   Racine         0


In [62]:
data_sample=[]
def restaurant_recommend_func(location,title):   
    global data_sample       
    global cosine_sim
    global sim_scores
    global tfidf_matrix
    global corpus_index
    global feature
    global rest_indices
    global idx
    
    # When location comes from function ,our new data consist only location dataset
    data_sample = df1.loc[df1['locality'] == location]  
    
    # index will be reset for cosine similarty index because Cosine similarty index has to be same value with result of tf-idf vectorize
    data_sample.reset_index(level=0, inplace=True) 
    
    #Feature Extraction
    data_sample['Split']="X"
    for i in range(0,data_sample.index[-1]):
        split_data=re.split(r'[,]', data_sample['cuisines'][i])
        for k,l in enumerate(split_data):
            split_data[k]=(split_data[k].replace(" ", ""))
        split_data=' '.join(split_data[:])
        data_sample['Split'].iloc[i]=split_data
        
    #TF-IDF vectorizer
    #Extracting Stopword
    tfidf = TfidfVectorizer(stop_words='english')
#Replace NaN for empty string
    data_sample['Split'] = data_sample['Split'].fillna('')
#Applying TF-IDF Vectorizer
    tfidf_matrix = tfidf.fit_transform(data_sample['Split'])
    tfidf_matrix.shape
    
    # Using for see Cosine Similarty scores
    feature= tfidf.get_feature_names()
#Cosine Similarity
    # Compute the cosine similarity matrix
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix) 
    
    # Column names are using for index
    corpus_index=[n for n in data_sample['Split']]
       
    #Construct a reverse map of indices    
    indices = pd.Series(data_sample.index, index=data_sample['name']).drop_duplicates() 
    
    #index of the restaurant matchs the cuisines
    idx = indices[title]
#Aggregate rating added with cosine score in sim_score list.
    sim_scores=[]
    for i,j in enumerate(cosine_sim[idx]):
        k=data_sample['rating_no'].iloc[i]
        if j != 0 :
            sim_scores.append((i,j,k))
            
    #Sort the restaurant names based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: (x[1],x[2]) , reverse=True)
# 10 similar cuisines
    sim_scores = sim_scores[0:10]
    rest_indices = [i[0] for i in sim_scores] 
  
    data_x =data_sample[['name','rating_no']].iloc[rest_indices]
    
    data_x['Cosine Similarity']=0
    for i,j in enumerate(sim_scores):
        data_x['Cosine Similarity'].iloc[i]=round(sim_scores[i][1],2)
   
    return data_x
# Top 10 similar restaurant with cuisine of 'Pizza Hut' restaurant in Connaught Place
restaurant_recommend_func('Racine',"Bernie's Pizzeria")

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,name,rating_no,Cosine Similarity
0,Infusino's Pizzeria,3.9,1.0
1,Bernie's Pizzeria,3.9,1.0
4,DeRango's The Pizza King,3.7,1.0
